### Boston Train Passenger Flow Prediction - YICHANG SHAO

In [1]:
# Third party libraries
import pandas as pd
import numpy as np
from sklearn import preprocessing
from pandas.tseries.holiday import USFederalHolidayCalendar

# input file path and output file path
file_path = '/Users/Eddy/Desktop/MBTA/Input/'
output_file_path = '/Users/Eddy/Desktop/MBTA/Output/'

#### Passenger Flow Data Processing I - Reading all MBTA passenger flow files

In [2]:
# Empty list
a = []

# Read and assign variables for every csv file can avoid touching off low memeory mode
d13 = pd.read_csv(file_path + 'gated_station_entries_2013.csv')
d14 = pd.read_csv(file_path + 'gated_station_entries_2014.csv')
d15 = pd.read_csv(file_path + 'gated_station_entries_2015.csv')
d16 = pd.read_csv(file_path + 'gated_station_entries_2016.csv')
d17 = pd.read_csv(file_path + 'gated_station_entries_2017.csv')
d18 = pd.read_csv(file_path + 'gated_station_entries_2018.csv')

# Merge csv files' data
a.append(d13)
a.append(d14)
a.append(d15)
a.append(d16)
a.append(d17)
a.append(d18)

# Declare dataframe
df = pd.concat(a, ignore_index=True)
df.head(10)

,STATION_NAME,GTFS_STOP_ID,DATE,TIME_PERIOD,STATION_ENTRIES
0,Back Bay,place-bbsta,2013-09-01,745,23
1,Back Bay,place-bbsta,2013-09-01,2200,38
2,Back Bay,place-bbsta,2013-09-01,2300,82
3,Back Bay,place-bbsta,2013-09-01,930,23
4,Back Bay,place-bbsta,2013-09-01,1245,89
5,Back Bay,place-bbsta,2013-09-01,1730,111
6,Back Bay,place-bbsta,2013-09-01,2000,94
7,Back Bay,place-bbsta,2013-09-01,2145,50
8,Back Bay,place-bbsta,2013-09-01,2415,48
9,Ashmont,place-asmnl,2013-09-01,1100,42


#### Passenger Flow Data Processing II - Calculating the whole day's passenger flow according date

In [3]:
# Using Airport station for example
station_data = df[df.STATION_NAME=='Airport']

# Creating pivot table, do the sum operation
station_data = station_data.pivot_table(index=['DATE'], values=['STATION_ENTRIES'], aggfunc=np.sum)
station_data.head(10)

,STATION_ENTRIES
DATE,
2013-09-01,10681
2013-09-02,7214
2013-09-03,7900
2013-09-04,7532
2013-09-05,7512
2013-09-06,7934
2013-09-07,14762
2013-09-08,11170
2013-09-09,7483


#### Holiday Data Processing

In [4]:
# Extracting US holidays from third party library directly
cal = USFederalHolidayCalendar()

# Choose year range we need
holidays = cal.holidays('2013-9', '2018')
holidays_series = pd.Series(1, index=holidays, name='HOLIDAY')

# Merge holiday data
station_data = station_data.join(holidays_series)

# Fill Nan value by 0
station_data['HOLIDAY'].fillna(0, inplace=True)
station_data.head(10)

,STATION_ENTRIES,HOLIDAY
DATE,,
2013-09-01,10681,0.0
2013-09-02,7214,1.0
2013-09-03,7900,0.0
2013-09-04,7532,0.0
2013-09-05,7512,0.0
2013-09-06,7934,0.0
2013-09-07,14762,0.0
2013-09-08,11170,0.0
2013-09-09,7483,0.0


#### Weather Data Processing

In [5]:
# Read historical weather data from csv file from website
raw_weather = pd.read_csv(file_path + 'weather.csv', index_col='DATE', parse_dates=True)

# Drop useless columns
raw_weather.drop(['STATION','TMAX', 'TMIN'], axis=1, inplace=True)

# Choose the date we want
weather = raw_weather['2013/9':'2018/12']

# Merge data
station_data = station_data.join(weather)
station_data.head(10)

,STATION_ENTRIES,HOLIDAY,PRCP,SNOW,TAVG
DATE,,,,,
2013-09-01,10681,0.0,0.38,0.0,73.0
2013-09-02,7214,1.0,0.38,0.0,73.0
2013-09-03,7900,0.0,0.00,0.0,75.0
2013-09-04,7532,0.0,0.00,0.0,72.0
2013-09-05,7512,0.0,0.09,0.0,67.0
2013-09-06,7934,0.0,0.00,0.0,61.0
2013-09-07,14762,0.0,0.00,0.0,66.0
2013-09-08,11170,0.0,0.00,0.0,70.0
2013-09-09,7483,0.0,0.00,0.0,61.0


#### Week Data Processing

In [6]:
# Week Data
days = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']
for i in range(7):
    station_data[days[i]] = (station_data.index.dayofweek == i).astype(int)

station_data.head(10)

,STATION_ENTRIES,HOLIDAY,PRCP,SNOW,TAVG,Mon,Tue,Wed,Thu,Fri,Sat,Sun
DATE,,,,,,,,,,,,
2013-09-01,10681,0.0,0.38,0.0,73.0,0,0,0,0,0,0,1
2013-09-02,7214,1.0,0.38,0.0,73.0,1,0,0,0,0,0,0
2013-09-03,7900,0.0,0.00,0.0,75.0,0,1,0,0,0,0,0
2013-09-04,7532,0.0,0.00,0.0,72.0,0,0,1,0,0,0,0
2013-09-05,7512,0.0,0.09,0.0,67.0,0,0,0,1,0,0,0
2013-09-06,7934,0.0,0.00,0.0,61.0,0,0,0,0,1,0,0
2013-09-07,14762,0.0,0.00,0.0,66.0,0,0,0,0,0,1,0
2013-09-08,11170,0.0,0.00,0.0,70.0,0,0,0,0,0,0,1
2013-09-09,7483,0.0,0.00,0.0,61.0,1,0,0,0,0,0,0


#### Month Data Processing

In [7]:
# Month Data
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
for i in range(12):
    station_data[months[i]] = (station_data.index.month == (i+1)).astype(int)

station_data.head()

,STATION_ENTRIES,HOLIDAY,PRCP,SNOW,TAVG,Mon,Tue,Wed,Thu,Fri,...,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
DATE,,,,,,,,,,,,,,,,,,,,,
2013-09-01,10681,0.0,0.38,0.0,73.0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2013-09-02,7214,1.0,0.38,0.0,73.0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2013-09-03,7900,0.0,0.00,0.0,75.0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2013-09-04,7532,0.0,0.00,0.0,72.0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
2013-09-05,7512,0.0,0.09,0.0,67.0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0


#### Data Normalization

In [8]:
# Remove data have large difference with normal passenger flow value, smaller than 2000 person/day
# Normal number is 7000 person/day
error_data = station_data[station_data['STATION_ENTRIES']<2000]
station_data = station_data[station_data['STATION_ENTRIES']>2000]

# Do data normalization, all input data should be in range from 0 to 1
min_max_scaler = preprocessing.MinMaxScaler()

station_data['PRCP'] = min_max_scaler.fit_transform(station_data[['PRCP']])
station_data['TAVG'] = min_max_scaler.fit_transform(station_data[['TAVG']])
station_data['SNOW'] = min_max_scaler.fit_transform(station_data[['SNOW']])

station_data.head(5)

,STATION_ENTRIES,HOLIDAY,PRCP,SNOW,TAVG,Mon,Tue,Wed,Thu,Fri,...,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
DATE,,,,,,,,,,,,,,,,,,,,,
2013-09-01,10681,0.0,0.131034,0.0,0.820225,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2013-09-02,7214,1.0,0.131034,0.0,0.820225,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2013-09-03,7900,0.0,0.000000,0.0,0.842697,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2013-09-04,7532,0.0,0.000000,0.0,0.808989,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
2013-09-05,7512,0.0,0.031034,0.0,0.752809,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0


#### Export Input Data

In [9]:
# Export the input data to csv file
input_data = pd.DataFrame(station_data, columns=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec','Mon','Tue','Wed','Thu','Fri','Sat','Sun', 'HOLIDAY', 'PRCP', 'SNOW', 'TAVG'])

input_data = input_data.T
input_data.to_csv(output_file_path + 'input.csv', index=None, header=None)

#### Export Output Data - Whole Day Output

In [10]:
# Export the output data to csv file
output_data_day =  pd.DataFrame(station_data['STATION_ENTRIES'])

output_data_day = output_data_day.T
output_data_day.to_csv(output_file_path + 'output_day.csv', index=None, header=None)

#### Show Error Data

In [11]:
error_data.index

DatetimeIndex(['2015-01-27', '2015-02-10', '2015-02-15', '2015-07-11',
               '2015-07-12', '2015-07-18', '2015-07-19', '2017-02-19',
               '2017-02-25', '2017-02-26', '2017-03-14', '2018-01-04',
               '2018-03-13', '2018-11-22'],
              dtype='datetime64[ns]', name='DATE', freq=None)

#### Export Output Data - Data Processing - Hour Period Output

In [12]:
station = df[df.STATION_NAME=='Airport']

# Declare bins
bins = [300,600,1200,1800,2400,2700]

# Give labels
labels = ['3AM-6AM','6AM-12AM','12AM-18PM','18PM-24PM','0AM-3AM']

# Cut the data
cats = pd.cut(station['TIME_PERIOD'], bins, labels=labels, right=False)
station.insert(5,'PERIOD',cats)

# Do sum for different time periods
station = station.pivot_table(index=['DATE', 'PERIOD'], values=['STATION_ENTRIES'], aggfunc=np.sum)
station.reset_index(inplace=True)
station.set_index('DATE', inplace=True)

for i in ['2015-01-27', '2015-02-10', '2015-02-15', '2015-07-11',
               '2015-07-12', '2015-07-18', '2015-07-19', '2017-02-19',
               '2017-02-25', '2017-02-26', '2017-03-14', '2018-01-04',
               '2018-03-13', '2018-11-22']:
    station.drop(i, inplace=True)

station.head(10)

,PERIOD,STATION_ENTRIES
DATE,,
2013-09-01,3AM-6AM,70
2013-09-01,6AM-12AM,4483
2013-09-01,12AM-18PM,4193
2013-09-01,18PM-24PM,1835
2013-09-01,0AM-3AM,100
2013-09-02,3AM-6AM,115
2013-09-02,6AM-12AM,2110
2013-09-02,12AM-18PM,2829
2013-09-02,18PM-24PM,2060


#### Export Output Data - Hour Period Output

In [13]:
# Export the output data to csv file
output_data_hour =  pd.DataFrame(station, columns=['PERIOD', 'STATION_ENTRIES'])

output_data_hour = output_data_hour.pivot_table(index=output_data_hour.index, columns=['PERIOD'], values=['STATION_ENTRIES'])
output_data_hour = output_data_hour.T
output_data_hour.to_csv(output_file_path + 'output_hour.csv', index=None, header=None)